In [ ]:
# todo:
# transmission terms as second order

## Import modules

In [ ]:
import numpy as np
import pandas as pd
from reactionmodel.model import Species
from reactionmodel.hook import HookAwareModel
import reactionmodel.parser as parser

## Load arrows from the excel document

In [ ]:
def concatenate_columns(df, columns, parentheses=True, sep='*', na_rep=None):
    df = df[columns].copy()
    if parentheses:
        df['left'] = '('
        df['right'] = ')'
    else:
        df['left'] = ''
        df['right'] = ''
    for c in columns:
        df[c] = df['left'].str.cat(df[c]).str.cat(df['right'])
    
    first_col = df[columns[0]]
    later_cols = df[columns[1:]]
    return first_col.str.cat(later_cols, sep=sep, na_rep=na_rep)

In [ ]:
arrows_file = pd.ExcelFile('./Shared model arrows.xlsx')
arrows = arrows_file.parse('Reactions').drop(0)
arrows = arrows[arrows['Include'].astype(bool)]
arrows['Initial'] = arrows['Initial (A)'].str.cat(arrows['Initial (N)'], sep='_')
arrows['Final'] = arrows['Final (A)'].str.cat(arrows['Final (N)'], sep='_')

arrows['Initial'] = arrows['Initial'].str.replace('-_-', '')
arrows['Final'] = arrows['Final'].str.replace('-_-', '')
# It's dumb, but k stem is guaranteed non-blank where Description (ie not Description stem) is guaranteed non-blank
arrows['k'] = concatenate_columns(arrows, ['k stem', 'k']).fillna(arrows['k stem'])
arrows['Description'] = concatenate_columns(arrows, ['Description stem', 'Description'], sep=' ', parentheses=False).fillna(arrows['Description'])

arrows = arrows[['Group', 'Description', 'Initial', 'Final', 'k', 'trigger']]

arrows

## Parse file into a Model

In [ ]:
# PROBLEM: DSDR can't exist in high abundance
families = {
    'star': ['M', 'X', 'DS', 'DR', 'DSDR']
}

In [ ]:
def parse_arrow(arrow):
    family_flag = False

    if pd.isna(arrow.trigger):
        arrow_dictionary = {'k': arrow.k}
    else:
        arrow_dictionary = {'p': arrow.k}

    reactants = arrow.Initial
    if reactants.count('*') > 1: assert False
    elif reactants.count('*') == 1: family_flag = True
    reactants = reactants.replace('*', '$i')

    products = arrow.Final
    if products.count('*') > 1: assert False
    elif products.count('*') == 1: family_flag = True
    products = products.replace('*', '$i')

    reactants = [reactants] if reactants else []
    products = [products] if products else []

    arrow_dictionary.update({
        'description': arrow.Description,
        'reactants': reactants,
        'products': products,
    })

    if family_flag:
        arrow_dictionary['used_families'] = {'i': 'star'}

    return arrow_dictionary

In [ ]:
from collections import defaultdict
reactions = []
triggered_sets = defaultdict(lambda: [])

for i, arrow in arrows.iterrows():
    destination = reactions
    if not pd.isna(arrow.trigger):
        destination = triggered_sets[arrow.Group]

    arrow_dictionary = parse_arrow(arrow)
    destination.append(arrow_dictionary)

In [ ]:
species_df = arrows_file.parse('Species')
base_species = [Species(s.Name, description=s.Description) for _,s in species_df.fillna('').iterrows()]

bad_species = [
    {'name':'DS_M'},
    {'name':'DS_X'},
    {'name':'DS_DR'},
    {'name':'DR_M'},
    {'name':'DR_X'},
    {'name':'DR_DS'},
    {'name':'X_M'},
    {'name':'DSDR_DSDR'},
    {'name':'DSDR_DS'},
    {'name':'DSDR_DR'},
]
bad_species = [Species(**s) for s in bad_species]

species = [s.to_dict() for s in base_species] + [s.to_dict() for s in bad_species]



In [ ]:
model_dictionary = {
    'families': families,
    'species': species,
    'reactions': reactions,
    'triggered_sets': triggered_sets
}

results = parser.loads(model_dictionary, model_class=HookAwareModel)

In [ ]:
filtered_reactions = [r for r in results.model.all_reactions if not set(r.reactants).intersection(bad_species) and not set(r.products).intersection(bad_species)]

In [ ]:
filtered_model = HookAwareModel(
    base_species,
    [r for r in results.model.all_reactions if not set(r.reactants).intersection(bad_species) and not set(r.products).intersection(bad_species)]
)

In [ ]:
parameters = arrows_file.parse('Parameters')[['Parameter', 'Value']].set_index('Parameter').to_dict()['Value']
## At some point here evaluate the rate constants at the parameters

needs_pressure_ds = [i for i,r in enumerate(filtered_model.all_reactions) if 'PRESSURE_DS' in r.k]
needs_pressure_dr = [i for i,r in enumerate(filtered_model.all_reactions) if 'PRESSURE_DR' in r.k]

parameters.update({'PRESSURE_DS': 1, 'PRESSURE_DR': 1})

all_k = filtered_model.get_k(parameters=parameters)
assert(isinstance(all_k, np.ndarray))

In [ ]:
import numpy as np
from reactionmodel.model import eval_expression

s_df = species_df.set_index('Name')
pressure_ds_vec = np.zeros(len(filtered_model.species))
pressure_dr_vec = np.zeros(len(filtered_model.species))

for i,s in enumerate(filtered_model.species):
    pressure_ds_vec[i] = eval_expression(str(s_df.loc[s.name]['PRESSURE_DS']), parameters)
    pressure_dr_vec[i] = eval_expression(str(s_df.loc[s.name]['PRESSURE_DR']), parameters)

In [ ]:
needs_pressure_ds

In [ ]:
needs_pressure_dr

In [ ]:
from numba import jit as numbajit

parametrized_reactions = []
for i,rxn in enumerate(filtered_model.all_reactions):
    k = all_k[i]
    if i in needs_pressure_ds:
        assert i not in needs_pressure_dr
        @numbajit(nopython=True)
        def pressure_ds(t, y):
            return np.sum(y * pressure_ds_vec)
        k = pressure_ds
    elif i in needs_pressure_dr:
        assert i not in needs_pressure_ds
        @numbajit(nopython=True)
        def pressure_dr(t, y):
            return np.sum(y * pressure_dr_vec)
        k = pressure_dr
    existing_dict = rxn.to_dict(keep_species_objects=True)
    existing_dict.update({'k': k})
    parametrized_reactions.append(type(rxn)(**existing_dict))

In [ ]:
parametrized_reactions = [r for r in parametrized_reactions if r.k != 0]

In [ ]:
len(filtered_model.all_reactions), len(parametrized_reactions)

In [ ]:
parametrized_model = HookAwareModel(
    base_species,
    parametrized_reactions
)

In [ ]:
parametrized_model.get_propensities_function(jit=False)